[범주형 변수](https://www.kaggle.com/code/alexisbcook/categorical-variables)  
이번 장에서는 숫자가 아닌 데이터들을 머신 러닝에 사용하는 방법을 배울 것이다.

이번 튜토리얼에서는 **범주형 데이터(Categorical Variable)**와 이를 다루는 세가지 방법을 배울 것이다.

# Introduction
**범주형 변수**는 제한된 숫자의 값만 사용한다.  

* 아침을 얼마나 자주 먹는지 네가지 옵션을 둔 설문 조사를 고려해보자(전혀/가끔/대부분/매일). 아래의 경우 응답이 고정된 범주로 속하기 때문에 데이터는 범주형이다.

* 사람들에게 어떤 브랜드의 자동차를 소유했는지 조사한다면, 응답은 '포드'/'혼다'/'토요타' 등의 범주형으로 대답할 것이다.   

이러한 범주형 변수를 전처러 없이 머신 러닝 모델에 연결하면 오류가 발생할 것이다. 이 튜토리얼에서는 범주형 데이터를 처리하는데 사용할 수 있는 세 가지 방법을 소개할 것이다. 

# Three Approaches

## Drop Categorical Variables 
범주형 변수를 다루는 가장 쉬운 방법은 그들을 데이터셋에서 지우는 것이다. 이 방법은 유용한 정보를 포함하지 않은 경우에만 잘 작동될 것이다. 

## Ordinal Encoding
Ordinal Encoding(순위 변환)은 각각의 고유 변수에 다른 값(수치)을 할당한다. 

![nn](https://i.imgur.com/tEogUAr.png![image.png](attachment:image.png))

이 방법은 범주에 순서를 가정한다.  
"Never"(0) < "Rarely"(1) < "Most Days"(2) < "Every Day"(3).

이 예제에서 범주마다 순위가 명확하기에 이러한 가정이 타당하다. 모든 범주형 변수가 값에 명확한 순서를 가지고 있는 것은 아니며 우리는 이를 **서수형 변수(ordinal variables)**라고 부른다. 트리 기반 모델(ex-dicision tree, randomforest)의 경우 순서 인코딩(ordinal encoding)이 순서 변수(ordinal variables)와 함께 잘 작동한다고 기대할 수 있다.

## One-Hot Encoding
**One Hot Encoding**은 원 데이터에서 값의 유뮤를 나타내는 새로운 열을 생성한다. (이를 이해하기 위해 예를 들어보자)  
![nn](https://i.imgur.com/TW5m0aJ.png![image.png](attachment:image.png)) 

기존 데이터에서 'Color'는 'Red'/'Yellow'/'Green' 세 가지 범주로 구성된 범주형 변수이다. One-Hot Encoding은 열에 각 범주와 행에 값의 유무를 포함한다. 원래 값이 'Red'인 경우 'Red' 열에 1을, 'Yellow'인 경우 'Yellow'열에 1을 넣었다. 

**ordinal encoding(서수 인코딩)**과 대조적으로, **one-hot encoding**은 범주의 순서를 가정하지 않는다. 따라서 범주형 데이터에 명확한 순서가 없는 경우(예:'빨간색'은 '노란색'보다 크거나 작지 않은 경우)이 방법이 특히 잘 작동된다고 기대할 수 있다. 우리는 내재적 순위가 없는 범주형 변수를 명목형 변수라 지칭한다.

범주형 변수가 많은 값을 갖는 경우(즉, 일반적으로 15개 이상의 값을 갖는 변수에
**One-Hot Encoding**은 일반적으로 범주형 변수가 많은 값을 갖는 경우 잘 수행되지 않는다.(15개 이상의 범주형 변수를 갖는 경우)

# Example
이전 튜토리얼과 마찬가지로 [Melbourne Housing dataset](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot)을 사용할 것이다. 

우리는 데이터 불러오기에 초점을 맞추지 않을 것이다. 대신, X_train, X_valid, y_train, y_valid에 이미 training 및 validation 데이터가 있는 지점에 있다고 상상할 수 있다. 

## 데이터 전처리

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('data/melb_data.csv')

# Separate target from predictors
y=data.Price
X=data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid=train_test_split(X, y, train_size=.8, random_state=0)

# Drop columns with missing vlaues (simplest approach)
cols_with_missing=[col for col in X_train_full.columns if X_train_full[col].isnull().any()]
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinality" means the number of unique values in a column
# "카디널리티"는 열의 고유한 값의 수를 의미한다.
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
# 카디널리티가 상대적으로 낮은 범주열을 선택하시오(편리하지만 임시적인 방법.)
low_cardinality_cols=[cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype=='object']

# Select numerical columns
numerical_cols=[cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64','float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [24]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.8272,4217.0


다음, 우리는 training data의 모든 범주형 변수 목록을 얻을 것이다.  
다음에서 각 열의 데이터 타입을 확인할 것이다. **object** dtype은 데이터에 텍스트가 있는 것을 나타낸다. (이론적으로 다른 것이 있을 수 있지만 그것은 현재 목적에 중요하지 않다.) 이 데이터 세트의 경우, text는 범주형 변수를 나타낸다. 

In [39]:
# Get list of categorical variable
s=(X_train.dtypes=='object')
object_cols=list(s[s].index)

print(f"\033[94m Categorical variables : ")
print(object_cols)

 Categorical variables : 
['Type', 'Method', 'Regionname']


## Define Function to Measure Quality of Each Approach
각각의 방법을 측정하기 위한 함수 정의  

---
우리는 범주형 변수를 다루는 세가지 방법을 비교하기 위해 **score_dataset()**을 정의할 것이다. 이 함수는 랜덤 포레스트 모델에서 **[평균 절대 오차(MAE)](https://en.wikipedia.org/wiki/Mean_absolute_error)**에 대해 보고한다.  
(일반적으로 MAE가 낮을 수록 성능이 좋다.)


In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model=RandomForestRegressor(n_estimators=100, random_state=100)
    model.fit(X_train, y_train)
    pred=model.predict(X_valid)
    
    return mean_absolute_error(y_valid, pred)

## 방법1.Drop Categorical Variables
**select_dtypes()** method를 통해 object 열을 삭제할 것이다.

In [43]:
drop_X_train=X_train.select_dtypes(exclude=['object'])
drop_X_valid=X_valid.select_dtypes(exclude=['object'])

print(f"\033[94m MAE from approach 1 (Drop Categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

 MAE from approach 1 (Drop Categorical variables):
177023.64449824087


## 방법2. Ordinal Encoding
Scikit-learn에는 순서 인코딩을 가져오는데 사용할 수 있는 **OrdinalEncoder** 클래스가 있다. 범주형 변수를 반복해 순서 인코더를 각 열에 개별적으로 적용한다. 

In [46]:
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data
label_X_train=X_train.copy()
label_X_valid=X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder=OrdinalEncoder()
label_X_train[object_cols]=ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols]=ordinal_encoder.transform(X_valid[object_cols])

print(f"\033[93m MAE from approach 2 (Ordinal Encoding):")
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

 MAE from approach 2 (Ordinal Encoding):
167190.81990339435


In [45]:
X_train

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,0.0,-37.85984,144.98670,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,193.0,-37.85800,144.90050,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,555.0,-37.79880,144.82200,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,265.0,-37.70830,144.91580,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,673.0,-37.76230,144.82720,4217.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13123,h,SP,Northern Metropolitan,3,5.2,3056.0,3.0,1.0,212.0,-37.77695,144.95785,11918.0
3264,h,S,Eastern Metropolitan,3,10.5,3081.0,3.0,1.0,748.0,-37.74160,145.04810,2947.0
9845,h,PI,Northern Metropolitan,4,6.7,3058.0,4.0,2.0,441.0,-37.73572,144.97256,11204.0
10799,h,S,Northern Metropolitan,3,12.0,3073.0,3.0,1.0,606.0,-37.72057,145.02615,21650.0


## 방법3. One-Hot Encoding

우리는 scikit-learn의 **OneHotEncoder** 클래스를 사용해 one-hot encoding을 얻는다. 해당 클래스에는 사용자가 동작을 지정하는 매개변수가 여러개 있다.  

* 훈련 데이터(training)에 검증 데이터(validation)가 있는 오류를 방지하기 위해 **handle_unknow='ignore'**을 설정했다. 
* **sparse=False**는 인코딩된 열이 숫자 배열로 반환되도록 보장한다.(소수 행렬의 경우)

encoder를 사용하기 위해 원핫 인코딩을 원하는 범주형 열만 제공한다. 예를 들어, train 데이터를 인코딩하기 위해 **X_train[object_cols]**를 제공한다.  
(object_cols는 범주형 데이터가 있는 열 이름 목록으로 X_train[object_cols]는 모든 범주형 데이터를 포함한다.)

In [49]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder=OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train=pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid=pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index=X_train.index
OH_cols_valid.index=X_valid.index

# Remove caegorical columns(will replace with one-hot encoding)
num_X_train=X_train.drop(object_cols, axis=1)
num_X_valid=X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train=pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid=pd.concat([num_X_valid, OH_cols_valid], axis=1)

print(f"\033[92m MAE from Approach 3 (One-Hot Encoding):")
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

 MAE from Approach 3 (One-Hot Encoding):


/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


167653.06074812397


/opt/homebrew/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


# 가장 좋은 방법은?
이 경우 범주형 열(**방법1**)을 제거하는 것이 MAE 점수가 가장 높기 때문에 성능이 가장 낮았다. 다른 두가지 방법의 MAE는 매우 근사한 값이기 때문에 서로 크게 다른 점이 없는 것으로 보인다.  

일반적으로 one-hot encoding **(방법3)**은 대체로 성능이 좋으며 **(방법1)**은 성능이 낮으나, 이는 경우에 따라 달라질 수 있다. 